# Ansatz Independent Shadow Optimization

This notebook contains code that can be used to simulate AISO in 8 qubit systems; 8 qubit VQSS problems and 4 qubit VQCS problems. All simulations are carried out as statevector simulation.

# Install Qiskit

Install Qiskit for sampling random Clifford circuits

In [3]:
!pip install qiskit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 10.7 MB/s eta 0:00:00


# Importing Packages

In [1]:
import numpy as np
from scipy.optimize import minimize
import time
import statistics
from qiskit.quantum_info import random_clifford

# Required functions

## Gates and Matrices

In [2]:
H_gate = (0.5 ** (0.5)) * np.matrix([[1,1],
               [1,-1]])

X_gate = np.matrix([[0 + 0j, 1 + 0j],
               [1 + 0j, 0+ 0j]])

Y_gate = np.matrix([[0 + 0j, -1j],
               [1j, 0 + 0j]])

Z_gate = np.matrix([[1 + 0j, 0 + 0j],
               [0 + 0j, -1 + 0j]])

Id = np.matrix([[1 + 0j, 0 + 0j],
                [0 + 0j, 1 + 0j]])

ket0_density = np.matrix([[1,0],
                          [0,0]])

ket1_density = np.matrix([[0,0],
                          [0,1]])

P_basis = (1/(2 ** 0.5)) * np.matrix([[1,0,1,0],
                     [0,1,0,-1j],
                     [0,1,0,1j],
                     [1,0,-1,0]])

P_basis_T = P_basis.T
P_basis_cT = np.conj(P_basis).T


P_basis2 = np.kron(P_basis, P_basis)
P_basis2_T = np.conj(P_basis2).T


Y_basis = (1/(2 ** 0.5)) * np.matrix([[1 + 0j, -1j],
                                      [1 + 0j, 1j]]) # This is the gate HS ^ \dagger

Y_basis_dag = np.conj(Y_basis).T


CNOT = np.reshape(np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]]), (2,2,2,2))


CNOT_matrix = np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]])

CZ = np.reshape(np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, -1 + 0j]]), (2,2,2,2))

CZ_matrix = np.matrix([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, -1 + 0j]])

S_gate = np.matrix([[1 + 0j, 0 + 0j],
                    [0 + 0j, 0 + 1j]])

S_dag_gate = np.matrix([[1 + 0j, 0 + 0j],
                    [0 + 0j, 0 - 1j]])



NOR_gate = np.zeros((2,2,2))
NOR_gate[0,0,1] = 1
NOR_gate[0,1,0] = 1
NOR_gate[1,0,0] = 1
NOR_gate[1,1,0] = 1

OR_gate = np.zeros((2,2,2))
OR_gate[0,0,0] = 1
OR_gate[0,1,1] = 1
OR_gate[1,0,1] = 1
OR_gate[1,1,1] = 1

OR_matrix = np.reshape(OR_gate, (4,2))

L_tensor = np.tensordot(OR_gate, OR_gate, axes = ([0], [2]))
L_tensor = np.tensordot(L_tensor, OR_gate, axes = ([0], [2]))
L_tensor = np.reshape(L_tensor, (2, 4, 4))


M_tensor = np.array([[[1, 0, 0],
                      [3.93234605, 0, 0],
                      [1.53313271, 1.20626572, 0]],
                     [[2.39842795, 1.43789702, 1.46774622],
                      [1.92640124, 2.19069889, 0.08802949],
                      [0.85882509, 0.26351743, -0.12805757]]])


SWAP = np.matrix([[1,0,0,0],
                 [0,0,1,0],
                 [0,1,0,0],
                 [0,0,0,1]])

SWAP_ID = np.kron(np.kron(Id, SWAP),Id)

X_gate_P = np.kron(X_gate, np.conj(X_gate))
X_gate_P = np.real(P_basis_T @ X_gate_P @ P_basis)
X_gate_P12 = np.kron(X_gate_P, np.eye(4))
X_gate_P21 = np.kron(np.eye(4), X_gate_P)

X_gate_12 = np.kron(X_gate, np.eye(2))
X_gate_21 = np.kron(np.eye(2), X_gate)

Y_gate_P = np.kron(Y_gate, np.conj(Y_gate))
Y_gate_P = np.real(P_basis_T @ Y_gate_P @ P_basis)
Y_gate_P12 = np.kron(Y_gate_P, np.eye(4))
Y_gate_P21 = np.kron(np.eye(4), Y_gate_P)

Y_gate_12 = np.kron(Y_gate, np.eye(2))
Y_gate_21 = np.kron(np.eye(2), Y_gate)

Z_gate_P = np.kron(Z_gate, np.conj(Z_gate))
Z_gate_P = np.real(P_basis_T @ Z_gate_P @ P_basis)
Z_gate_P12 = np.kron(Z_gate_P, np.eye(4))
Z_gate_P21 = np.kron(np.eye(4), Z_gate_P)

Z_gate_12 = np.kron(Z_gate, np.eye(2))
Z_gate_21 = np.kron(np.eye(2), Z_gate)

H_gate_P = np.kron(H_gate, np.conj(H_gate))
H_gate_P = np.real(P_basis_T @ H_gate_P @ P_basis)
H_gate_P12 = np.kron(H_gate_P, np.eye(4))
H_gate_P21 = np.kron(np.eye(4), H_gate_P)

H_gate_12 = np.kron(H_gate, np.eye(2))
H_gate_21 = np.kron(np.eye(2), H_gate)

S_gate_P = np.kron(S_gate, np.conj(S_gate))
S_gate_P = np.real(P_basis_T @ S_gate_P @ P_basis)
S_gate_P12 = np.kron(S_gate_P, np.eye(4))
S_gate_P21 = np.kron(np.eye(4), S_gate_P)

S_gate_12 = np.kron(S_gate, np.eye(2))
S_gate_21 = np.kron(np.eye(2), S_gate)

S_dag_gate_P = np.kron(S_dag_gate, np.conj(S_dag_gate))
S_dag_gate_P = np.real(P_basis_T @ S_dag_gate_P @ P_basis)
S_dag_gate_P12 = np.kron(S_dag_gate_P, np.eye(4))
S_dag_gate_P21 = np.kron(np.eye(4), S_dag_gate_P)

S_dag_gate_12 = np.kron(S_dag_gate, np.eye(2))
S_dag_gate_21 = np.kron(np.eye(2), S_dag_gate)

Y_basis_P = np.kron(Y_basis, np.conj(Y_basis))
Y_basis_P = np.real(P_basis_T @ Y_basis_P @ P_basis)
Y_basis_P12 = np.kron(Y_basis_P, np.eye(4))
Y_basis_P21 = np.kron(np.eye(4), Y_basis_P)

Y_basis_dag_P = np.kron(Y_basis_dag, np.conj(Y_basis_dag))
Y_basis_dag_P = np.real(P_basis_T @ Y_basis_dag_P @ P_basis)
Y_basis_dag_P12 = np.kron(Y_basis_dag_P, np.eye(4))
Y_basis_dag_P21 = np.kron(np.eye(4), Y_basis_dag_P)

CNOT_12_big = np.matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]).T

CNOT_12_P = P_basis2_T @ CNOT_12_big @ P_basis2
CNOT_21_big = np.matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]).T

CNOT_21_P = np.real(P_basis2_T @ CNOT_21_big @ P_basis2)

CNOT_12 = np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j]])

CNOT_21 = np.array([[1 + 0j, 0 + 0j, 0 + 0j, 0 + 0j],
                  [0 + 0j, 0 + 0j, 0 + 0j, 1 + 0j],
                  [0 + 0j, 0 + 0j, 1 + 0j, 0 + 0j],
                  [0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j]])

clifford_p_basis1 = {'X': X_gate_P,
                    'Y': Y_gate_P,
                    'Z': Z_gate_P,
                    'H': H_gate_P,
                    'S': S_gate_P,
                    'S_dag': S_dag_gate_P, 'Y_basis': Y_basis_P, 'Y_basis_dag': Y_basis_dag_P}

clifford_p_basis2 = {'X': [X_gate_P12, X_gate_P21],
                    'Y': [Y_gate_P12, Y_gate_P21],
                    'Z': [Z_gate_P12, Z_gate_P21],
                    'H': [H_gate_P12, H_gate_P21],
                    'S': [S_gate_P12, S_gate_P21],
                    'S_dag': [S_dag_gate_P12, S_dag_gate_P21],
                    'CX': [CNOT_12_P, CNOT_21_P]}

clifford_gate_dict = {'H': H_gate, 'S': S_gate, 'X':X_gate, 'Y': Y_gate, 'Z':Z_gate,
                      'CX': CNOT, 'Y_basis': Y_basis, 'Y_basis_dag': Y_basis_dag, 'I': Id, 'S_dag': S_dag_gate}

clifford_gate_dict2 = {'X': [X_gate_12, X_gate_21],
                    'Y': [Y_gate_12, Y_gate_21],
                    'Z': [Z_gate_12, Z_gate_21],
                    'H': [H_gate_12, H_gate_21],
                    'S': [S_gate_12, S_gate_21],
                    'S_dag': [S_dag_gate_12, S_dag_gate_21],
                    'CX': [CNOT_12, CNOT_21]}

# This function returns a numpy matrix of the form R_Z(theta) for any real number theta
def R_Z(theta):
  return np.matrix([[np.exp(-1j * theta/2), 0],[0, np.exp(1j * theta/2)]])

# This function returns a numpy matrix of the form R_Y(theta) for any real number theta
def R_Y(theta):
  return np.matrix([[np.cos(theta/2), (-1 * np.sin(theta/2))], [np.sin(theta/2), np.cos(theta/2)]])

# This function returns a numpy matrix of the form R_X(theta) for any real number theta
def R_X(theta):
  return np.matrix([[np.cos(theta/2), ((-1j) * np.sin(theta/2))], [((-1j) * np.sin(theta/2)), np.cos(theta/2)]])

def CNOT(is_rev=0):
  if is_rev == 0:
    return CNOT_12

  else:
    return CNOT_21

Id4 = np.eye(4)

# This function returns a matrix of the subcircuit s_gate.
# 1, theta: A 2 dimensional numpy array
def s_gate(theta):
  return np.kron(R_X(theta[0]), R_Y(theta[1])) @ CNOT()


# Inverse of above gate
def s_gate_inverse(theta):
  return CNOT() @ np.kron(R_X(-theta[0]), R_Y(-theta[1]))

# This function returns a matrix which is built up of 2 subcircuit s_gates.
# 1, theta: A 2 dimensional numpy array
def s_gate_double(theta):
  return np.kron(R_X(theta[0]), R_Y(theta[1])) @ CNOT() @ np.kron(R_X(theta[2]), R_Y(theta[3])) @ CNOT()

# Inverse of above gate
def s_gate_double_inverse(theta):
  return CNOT() @ np.kron(R_X(-theta[2]), R_Y(-theta[3])) @ CNOT() @ np.kron(R_X(-theta[0]), R_Y(-theta[1]))



SWAP_P = SWAP_ID @ P_basis2
SWAP_P_T = np.conj(SWAP_P).T
LM = np.tensordot(L_tensor, M_tensor, axes = ([0], [0]))
LM_reshaped = np.reshape(LM, (16, 3, 3))

# This function converts a gate is std basis to pauli basis
# gate: numpy matrix
# n_qubits_to_act: number of qubits that gate acts on
def convert_to_pauli_basis(gate, n_qubits_to_act = 2):
  if n_qubits_to_act == 2:
    return np.real(SWAP_P_T @ np.kron(gate, np.conj(gate)) @ SWAP_P)

  else:
    gate_P = np.kron(gate, np.conj(gate))
    return np.real(P_basis_cT @ gate_P @ P_basis)


bell_gate = CNOT_matrix @ np.kron(H_gate, Id)
bell_gate_inv = np.kron(H_gate, Id) @ CNOT_matrix

bell_gate_P = convert_to_pauli_basis(bell_gate)
bell_gate_inv_P = convert_to_pauli_basis(bell_gate_inv)

## Basic functions

In [3]:
# This function applues a gate of a specified qubit of a state.
# 1. state: 8 mode numpy tensor 
# 2. qubits_to_act: list of qubits to act
# 3: gate: numpy matrix
def apply_gate(state, qubits_to_act, gate):

  state = np.tensordot(state, np.reshape(np.array(gate), tuple([state.shape[0]] * (2 * len(qubits_to_act)))), axes = [qubits_to_act, list(range(-len(qubits_to_act),0))])
  state = np.moveaxis(state, list(range(-len(qubits_to_act),0)), qubits_to_act)
  return state

# This function is used to convert an integer to a list of binary values.
# 1. x: integer value
# 2. BitNo: how many binary bits for x's expansion is required
def binary(x, BitNo):
  format(x, 'b').zfill(BitNo)
  Binlist = [int(y) for y in list(format(x, 'b').zfill(BitNo))]
  return Binlist

# This function converts a state to a vector of amplitudes
def convert_to_amplitudes(state):
  return np.abs(state.flatten()) ** 2

# This function generates a uniformly randomly generated 2 qubit Clifford gate
def generate_random_clifford_circuit_qiskit():
  return np.matrix(random_clifford(2, seed=None).to_operator())

# This function returns the probability of the state measuring all 0s
# 1. state: 8 mode tensor
# 2. no_of_shots: Number of shots of measurement
def compute_expectation_all0_directly(state, no_of_shots = 0):
  if no_of_shots == 0:
    return np.abs(state[0,0,0,0,0,0,0,0]) ** 2

  else:
    pr = np.abs(state[0,0,0,0,0,0,0,0]) ** 2
    return np.mean(np.random.choice([1,0], no_of_shots, p = [pr, 1-pr]))

## Shadow related

In [4]:
# This function generates shallow shadows
# 1. state: 8 mode tensor
# 2. depth: depth of shadow generating ensemble
# 3. T: total number of shadows
# 4. interval: interval at which a print should occur denoting the index of the shadow
def generate_shallow_shadow(state, depth, T, interval = 1000):
  n_qubits = len(state.shape)

  shadow_state = np.zeros(tuple([4] * n_qubits))

  for j in range(T):
    if (j+1) % interval == 0:
      print("Curernt shadow: ", j)
    temp_state = np.copy(state)
    gate_list2 = []
    for d in range(depth):
      for i in range(0, n_qubits, 2):
        gate = generate_random_clifford_circuit_qiskit()

        gate_list2.append((convert_to_pauli_basis(gate), [i + (d % 2), (i + 1 + (d % 2)) % n_qubits]))
        temp_state = apply_gate(temp_state, [i + (d % 2), (i + 1 + (d % 2)) % n_qubits], gate)

    prob_array = convert_to_amplitudes(temp_state)
    outcome = np.random.choice(2 ** n_qubits, 1, p = list(prob_array))[0]
    outcome_bin = binary(outcome, n_qubits)

    final_state = np.array([1])
    for i in range(len(outcome_bin)):
      if outcome_bin[i] == 0:
        final_state = np.kron(final_state, np.array([1/(2 ** 0.5), 0, 1/(2 ** 0.5), 0]))
      else:
        final_state = np.kron(final_state, np.array([1/(2 ** 0.5), 0, -1/(2 ** 0.5), 0]))

    count = len(gate_list2)-1
    final_state = np.reshape(final_state, tuple([4] * n_qubits))
    for d in range(depth):
      for i in range(int(n_qubits/2)):

        final_state = apply_gate(final_state, gate_list2[count][1], np.conj(gate_list2[count][0]).T)
        count -= 1

    shadow_state += final_state/T
  return shadow_state.flatten()



def compute_expectation_entries(mps):
  for i in range(0, len(mps), 2):
    temp = np.tensordot(mps[i], mps[i+1], axes = ([2], [1]))
    temp = np.tensordot(temp, LM, axes = ([0, 2], [0, 1]))

    temp = np.ndarray.transpose(temp, [0,2,1,3])
    temp = np.reshape(temp, (temp.shape[0] * temp.shape[1], temp.shape[2] * temp.shape[3]))
    if i == 0:
      ans_mat = temp

    else:
      ans_mat = ans_mat @ temp
  return np.trace(ans_mat)


# Using th3 above function, this function computes a single entry of the shadow generating inverse operator
def compute_entries(pos):
    ans_list = []
    q = pos
    for div in range(8):
      if q != 0:
        r = q % 4
        ans_list.append(np.reshape(np.eye(4)[r], (4,1,1)))
        q = int(q/4)
      else:
        ans_list.append(np.reshape(np.eye(4)[0], (4,1,1)))

    ans_list = list(reversed(ans_list))

    return compute_expectation_entries(ans_list)


# This function returns the shadow generating inverse operator
def generate_inv_matrix():
  ans = []
  for index in range(4 ** 8):
    if index % 10000 == 0:
      print(index)
    ans.append(compute_entries(index))

  return np.array(ans)

# This function returns the all 0 density matrix in pauli basis
def create_all0_obs(n_qubits):
    obs = np.array([1])
    for i in range(n_qubits):
        obs = np.kron(obs, np.array([1/(2 ** 0.5),0,1/(2 ** 0.5),0]))

    return np.reshape(obs, tuple([4] * n_qubits))

# This function returns the maximally entangled state in pauli basis
def maximally_entangled_pauli():
    obs = create_all0_obs(8)
    for i in range(4):
        obs = apply_gate(obs, [i, i+4], bell_gate_P)
    return obs


# This function applies a maximally entangling operation of a given state
# 1. state: 8 mode numpy tensor
# 2. is_pauli: set it to 1 to apply in pauli basis
def entangle(state, is_pauli = 0):
  n_qubits = len(state.shape)
  if is_pauli == 0:
    for i in range(int(n_qubits/2)):
      state = apply_gate(state, [i, i+int(n_qubits/2)], bell_gate)

  else:
    for i in range(int(n_qubits/2)):
      state = apply_gate(state, [i, i+int(n_qubits/2)], bell_gate_P)

  return state

# This function applies the inverse of a maximally entangling operation of a given state
# 1. state: 8 mode numpy tensor
# 2. is_pauli: set it to 1 to apply in pauli basis
def disentangle(state, is_pauli = 0):
  n_qubits = len(state.shape)
  if is_pauli == 0:
    for i in range(int(n_qubits/2)):
      state = apply_gate(state, [i, i+int(n_qubits/2)], bell_gate_inv)

  else:
    for i in range(int(n_qubits/2)):
      state = apply_gate(state, [i, i+int(n_qubits/2)], bell_gate_inv_P)

  return state

# This function computes the inner product between a shadow and an obs. obs should be given as 8 mode tensor.
def shadow_expectation_naive(shadow_state, obs):
    return np.dot(shadow_state, obs.flatten())


## VQSS Circuits and Optimization

In [5]:

# These functions apply the corresponding circuit to the state
def ala(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  depth = int(np.log2(n_qubits))
  pos = 0
  len_s = 2
  for d in range(depth):
    for i in range(0, n_qubits, 2):
      if is_pauli == 0:
        gate = s_gate_inverse(theta[pos * len_s: (pos+1) * len_s])
      else:
        gate = convert_to_pauli_basis(s_gate_inverse(theta[pos * len_s: (pos+1) * len_s]))
      temp_state = apply_gate(temp_state, [i + (d % 2), ((i + 1 + (d % 2)) % n_qubits)], gate)
      pos += 1

  return temp_state


def ala_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  depth = int(np.log2(n_qubits))
  len_s = 2
  pos = int(len(theta)/len_s)
  for d in range(depth-1, -1, -1):
    for i in range(n_qubits-2, -1, -2):
      if is_pauli == 0:
        gate = s_gate(theta[(pos-1) * len_s: pos* len_s])
      else:
        gate = convert_to_pauli_basis(s_gate(theta[(pos-1) * len_s: pos* len_s]))
      temp_state = apply_gate(temp_state, [i + (d % 2), ((i + 1 + (d % 2)) % n_qubits)], gate)
      pos -= 1

  return temp_state



def ttn(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = int(len(theta)/(n_qubits-1))
  k = 0
  for d in range(3):
    for i in range(2 ** d):
      if is_pauli == 0:
        gate = s_gate_inverse(theta[len_s * k: len_s * (k+1)])
      else:
        gate = convert_to_pauli_basis(s_gate_inverse(theta[len_s * k: len_s * (k+1)]))
      temp_state = apply_gate(temp_state, [(2 ** (3-d)) * i, ((2 ** (3-d)) * (i+1))-1], gate)
      k += 1

  return temp_state


def ttn_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = int(len(theta)/(n_qubits-1))
  k = n_qubits-2
  for d in range(2,-1,-1):
    for i in range((2 ** d)-1, -1, -1):
      if is_pauli == 0:
        gate = s_gate(theta[len_s * k: len_s * (k+1)])
      else:
        gate = convert_to_pauli_basis(s_gate(theta[len_s * k: len_s * (k+1)]))
      temp_state = apply_gate(temp_state, [(2 ** (3-d)) * i, ((2 ** (3-d)) * (i+1))-1], gate)
      k -= 1

  return temp_state


def mera(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = 2
  k = 0
  for d in range(3):
    if d!=0:
      for i in range(2 ** d):
        if is_pauli == 0:
          gate = s_gate_inverse(theta[len_s * k: len_s * (k+1)])
        else:
          gate = convert_to_pauli_basis(s_gate_inverse(theta[len_s * k: len_s * (k+1)]))
        temp_state = apply_gate(temp_state, [((2 ** (3-d)) * i) + 1, ((((2 ** (3-d)) * (i+1))-1) + 1) % n_qubits], gate)
        k += 1


    for i in range(2 ** d):
      if is_pauli == 0:
        gate = s_gate_inverse(theta[len_s * k: len_s * (k+1)])

      else:
        gate = convert_to_pauli_basis(s_gate_inverse(theta[len_s * k: len_s * (k+1)]))
      temp_state = apply_gate(temp_state, [(2 ** (3-d)) * i, ((2 ** (3-d)) * (i+1))-1], gate)
      k += 1

  return temp_state



def mera_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = 2
  k = 12
  for d in range(2,-1,-1):
    for i in range((2 ** d)-1, -1, -1):
      if is_pauli == 0:
        gate = s_gate(theta[len_s * k: len_s * (k+1)])
      else:
        gate = convert_to_pauli_basis(s_gate(theta[2 * k: 2 * (k+1)]))
      temp_state = apply_gate(temp_state, [(2 ** (3-d)) * i, ((2 ** (3-d)) * (i+1))-1], gate)
      k -= 1

    if d != 0:
      for i in range((2 ** d)-1, -1, -1):
        if is_pauli == 0:
          gate = s_gate(theta[len_s * k: len_s * (k+1)])

        else:
          gate = convert_to_pauli_basis(s_gate(theta[len_s * k: len_s * (k+1)]))
        temp_state = apply_gate(temp_state, [((2 ** (3-d)) * i) + 1, ((((2 ** (3-d)) * (i+1))-1) + 1) % n_qubits], gate)
        k -= 1

  return temp_state

def hea(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  depth = 3

  pos = 0
  for d in range(depth):
    for i in range(n_qubits):
      if i % 2 == 0:
        if is_pauli == 0:
          gate = R_X(theta[pos])
        else:
          gate = convert_to_pauli_basis(R_X(theta[pos]), 1)
        temp_state = apply_gate(temp_state, [i], gate)
        pos += 1

      else:
        if is_pauli == 0:
          gate = R_Y(theta[pos])
        else:
          gate = convert_to_pauli_basis(R_Y(theta[pos]), 1)
        temp_state = apply_gate(temp_state, [i], gate)
        pos += 1

    for i in range(0, n_qubits, 2):
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT())
      else:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT_12_P)

    for i in range(1, n_qubits-1, 2):
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT())
      else:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT_12_P)

    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [7, 0], CNOT())
    else:
      temp_state = apply_gate(temp_state, [7, 0], CNOT_12_P)
  return temp_state


def hea_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  depth = 3

  pos = 23
  for d in range(depth):
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [7, 0], CNOT())
    else:
      temp_state = apply_gate(temp_state, [7, 0], CNOT_12_P)

    for i in range(1, n_qubits-1, 2):
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT())
      else:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT_12_P)

    for i in range(0, n_qubits, 2):
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT())
      else:
        temp_state = apply_gate(temp_state, [i, i+1], CNOT_12_P)

    for i in range(n_qubits-1,-1,-1):
      if i % 2 == 0:
        if is_pauli == 0:
          gate = R_X(-theta[pos])
        else:
          gate = convert_to_pauli_basis(R_X(-theta[pos]), 1)
        temp_state = apply_gate(temp_state, [i], gate)
        pos -= 1

      else:
        if is_pauli == 0:
          gate = R_Y(-theta[pos])
        else:
          gate = convert_to_pauli_basis(R_Y(-theta[pos]), 1)
        temp_state = apply_gate(temp_state, [i], gate)
        pos -= 1

  return temp_state

#-----------------------------------------------------------------------------------------------------------------

# This function carries out one step of SPSA with AISO
def single_spsa_update_aiso_vqss(shadow_state, theta, ansatz, obs, c_alpha, a_alpha, idx):
  delta = np.random.choice([-1, 1], len(theta), p = [1/2] * 2)

  c = 1/(idx ** c_alpha)
  a = 1/(idx ** a_alpha)

  if ansatz == 'ala':
    obs_plus = ala_inverse(theta + (c * delta), obs, 1)
    obs_minus = ala_inverse(theta - (c * delta), obs, 1)

  elif ansatz == 'ttn':
    obs_plus = ttn_inverse(theta + (c * delta), obs, 1)
    obs_minus = ttn_inverse(theta - (c * delta), obs, 1)

  elif ansatz == 'hea':
    obs_plus = hea_inverse(theta + (c * delta), obs, 1)
    obs_minus = hea_inverse(theta - (c * delta), obs, 1)

  elif ansatz == 'mera':
    obs_plus = mera_inverse(theta + (c * delta), obs, 1)
    obs_minus = mera_inverse(theta - (c * delta), obs, 1)

  g = (shadow_expectation_naive(shadow_state, obs_plus) - shadow_expectation_naive(shadow_state, obs_minus))/(2 * c)

  return theta + (a * g * delta)

# This function carries out SPSA with AISO
def aiso_vqss(theta, state, shadow_state, ansatz, c_alpha, a_alpha, no_of_iter, interval = 100):
    iter_no = 1
    infid_list = []
    theta_list = []
    temp_state = np.copy(state)
    obs = create_all0_obs(n_qubits)
    while iter_no < no_of_iter + 1:

        theta = single_spsa_update_aiso_vqss(shadow_state, theta, ansatz, obs, c_alpha, a_alpha, iter_no)
        if iter_no % interval == 0:

            init_state = np.copy(state)
            if ansatz == 'ala':
              fidelity = compute_expectation_all0_directly(ala(theta, init_state))
            elif ansatz == 'ttn':
              fidelity = compute_expectation_all0_directly(ttn(theta, init_state))
            elif ansatz == 'mera':
              fidelity = compute_expectation_all0_directly(mera(theta, init_state))
            elif ansatz == 'hea':
              fidelity = compute_expectation_all0_directly(hea(theta, init_state))

            infid_list.append(1-fidelity)
            theta_list.append(theta)
        if iter_no % 100 == 0:
            print("Current iteration: {}".format(iter_no))
            print("Current Infidelity: {}".format(1-fidelity))
        iter_no += 1
    return infid_list, theta_list

# This function carries out one step of SPSA with VQA
def single_spsa_update_vqa_vqss(state, ansatz, theta, c_alpha, a_alpha, idx, no_of_shots = 0):
  n_qubits = len(state.shape)
  delta = np.random.choice([-1, 1], len(theta), p = [1/2] * 2)

  c = 1/(idx ** c_alpha)
  a = 1/(idx ** a_alpha)


  if ansatz == 'ala':
    state_plus = ala(theta + (c * delta), state)
    state_minus = ala(theta - (c * delta), state)

  elif ansatz == 'ttn':
    state_plus = ttn(theta + (c * delta), state)
    state_minus = ttn(theta - (c * delta), state)

  elif ansatz == 'mera':
    state_plus = mera(theta + (c * delta), state)
    state_minus = mera(theta - (c * delta), state)

  elif ansatz == 'hea':
    state_plus = hea(theta + (c * delta), state)
    state_minus = hea(theta - (c * delta), state)

  exp_plus = compute_expectation_all0_directly(state_plus, no_of_shots)
  exp_minus = compute_expectation_all0_directly(state_minus, no_of_shots)

  g = (exp_plus - exp_minus)/(2 * c)

  return theta + (a * g * delta)

# This function carries out SPSA with VQA
def vqa_vqss(theta, state, ansatz, c_alpha, a_alpha, no_of_iter, interval = 100, no_of_shots = 0):
    iter_no = 1
    n_qubits = len(state.shape)
    infid_list = []
    theta_list = []
    temp_state = np.copy(state)
    while iter_no < no_of_iter + 1:

        theta = single_spsa_update_vqa_vqss(temp_state, ansatz, theta, c_alpha, a_alpha, iter_no, no_of_shots = no_of_shots)
        if iter_no % interval == 0:

            init_state = np.copy(state)
            if ansatz == 'ala':
              init_state = ala(theta, init_state)
            elif ansatz == 'ttn':
              init_state = ttn(theta, init_state)
            elif ansatz == 'mera':
              init_state = mera(theta, init_state)
            elif ansatz == 'hea':
              init_state = hea(theta, init_state)

            fidelity = compute_expectation_all0_directly(init_state)

            infid_list.append(1-fidelity)
            theta_list.append(theta)
        if iter_no % 100 == 0:
            print("Current iteration: {}".format(iter_no))
            print("Current Infidelity: {}".format(1-fidelity))
        iter_no += 1
    return infid_list, theta_list

## VQCS Circuits and Optimization

In [6]:

# These functions apply the corresponding circuit to the circuit-state
def ala_circ(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  depth = int(np.log2(n_qubits/2))
  pos = 0
  len_s = 4
  for d in range(depth):
    for i in range(0, int(n_qubits/2), 2):
      if is_pauli == 0:
        gate = s_gate_double(theta[pos * len_s: (pos+1) * len_s])
      else:
        gate = convert_to_pauli_basis(s_gate_double(theta[pos * len_s: (pos+1) * len_s]))
      temp_state = apply_gate(temp_state, [i + (d % 2) + int(n_qubits/2), ((i + 1 + (d % 2)) % int(n_qubits/2)) + int(n_qubits/2)], gate)
      pos += 1

  return temp_state


def ala_circ_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  depth = int(np.log2(n_qubits/2))
  len_s = 4
  pos = int(len(theta)/len_s)
  for d in range(depth-1, -1, -1):
    for i in range(int(n_qubits/2)-2, -1, -2):
      if is_pauli == 0:
        gate = s_gate_double_inverse(theta[(pos-1) * len_s: pos* len_s])
      else:
        gate = convert_to_pauli_basis(s_gate_double_inverse(theta[(pos-1) * len_s: pos* len_s]))
      temp_state = apply_gate(temp_state, [i + (d % 2) + int(n_qubits/2), ((i + 1 + (d % 2)) % int(n_qubits/2)) + int(n_qubits/2)], gate)
      pos -= 1

  return temp_state


def mera_circ(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = 4
  k = 4
  if is_pauli == 0:
    gate = s_gate_double(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [5,7], gate)
  k -= 1

  if is_pauli == 0:
    gate = s_gate_double(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [7,4], gate)
  k -= 1

  if is_pauli == 0:
    gate = s_gate_double(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [5,6], gate)
  k -= 1

  if is_pauli == 0:
    gate = s_gate_double(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [6,7], gate)
  k -= 1

  if is_pauli == 0:
    gate = s_gate_double(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [4,5], gate)
  k -= 1

  return temp_state


def mera_circ_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = 4
  k = 0

  if is_pauli == 0:
    gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [4,5], gate)
  k += 1

  if is_pauli == 0:
    gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [6,7], gate)
  k += 1

  if is_pauli == 0:
    gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [5,6], gate)
  k += 1

  if is_pauli == 0:
    gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [7,4], gate)
  k += 1

  if is_pauli == 0:
    gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])
  else:
    gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  temp_state = apply_gate(temp_state, [5,7], gate)
  k += 1

  return temp_state


def hea_circ(theta, state, is_pauli = 0):
  temp_state = np.copy(state)
  depth = 2

  pos = 0

  for d in range(2):
    gate = R_X(theta[pos+1]) @ R_Z(theta[pos])
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [4], gate)
    else:
      temp_state = apply_gate(temp_state, [4], convert_to_pauli_basis(gate,1))
    pos += 2

    gate = R_Y(theta[pos+1]) @ R_Z(theta[pos])
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [5], gate)
    else:
      temp_state = apply_gate(temp_state, [5], convert_to_pauli_basis(gate,1))
    pos += 2
    
    gate = R_X(theta[pos+1]) @ R_Z(theta[pos])
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [6], gate)
    else:
      temp_state = apply_gate(temp_state, [6], convert_to_pauli_basis(gate,1))
    pos += 2

    gate = R_Y(theta[pos+1]) @ R_Z(theta[pos])
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [7], gate)
    else:
      temp_state = apply_gate(temp_state, [7], convert_to_pauli_basis(gate,1))
    pos += 2


    gate = CNOT()
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [4,5], CNOT())
    else:
      temp_state = apply_gate(temp_state, [4,5], convert_to_pauli_basis(CNOT()))

    gate = CNOT()
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [6,7], CNOT())
    else:
      temp_state = apply_gate(temp_state, [6,7], convert_to_pauli_basis(CNOT()))

    gate = CNOT()
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [5,6], CNOT())
    else:
      temp_state = apply_gate(temp_state, [5,6], convert_to_pauli_basis(CNOT()))

    gate = CNOT()
    if is_pauli == 0:
      temp_state = apply_gate(temp_state, [7,0], CNOT())
    else:
      temp_state = apply_gate(temp_state, [7,0], convert_to_pauli_basis(CNOT()))

  return temp_state

def hea_circ_inverse(theta, state, is_pauli = 0):
    temp_state = np.copy(state)
    depth = 2

    pos = 15

    for d in range(2):
      gate = CNOT()
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [7,0], CNOT())
      else:
        temp_state = apply_gate(temp_state, [7,0], convert_to_pauli_basis(CNOT()))

      gate = CNOT()
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [5,6], CNOT())
      else:
        temp_state = apply_gate(temp_state, [5,6], convert_to_pauli_basis(CNOT()))

      gate = CNOT()
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [6,7], CNOT())
      else:
        temp_state = apply_gate(temp_state, [6,7], convert_to_pauli_basis(CNOT()))

      gate = CNOT()
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [4,5], CNOT())
      else:
        temp_state = apply_gate(temp_state, [4,5], convert_to_pauli_basis(CNOT()))

      
      gate =  R_Z(-theta[pos-1]) @ R_Y(-theta[pos])
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [7], gate)
      else:
        temp_state = apply_gate(temp_state, [7], convert_to_pauli_basis(gate,1))
      pos -= 2

      gate =  R_Z(-theta[pos-1]) @ R_X(-theta[pos])
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [6], gate)
      else:
        temp_state = apply_gate(temp_state, [6], convert_to_pauli_basis(gate,1))
      pos -= 2

      gate =  R_Z(-theta[pos-1]) @ R_Y(-theta[pos])
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [5], gate)
      else:
        temp_state = apply_gate(temp_state, [5], convert_to_pauli_basis(gate,1))
      pos -= 2


      gate =  R_Z(-theta[pos-1]) @ R_X(-theta[pos])
      if is_pauli == 0:
        temp_state = apply_gate(temp_state, [4], gate)
      else:
        temp_state = apply_gate(temp_state, [4], convert_to_pauli_basis(gate,1))
      pos -= 2

    return temp_state

def ttn_circ_inverse(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = 4
  k = 0

  if is_pauli == 0:
    temp_state = apply_gate(temp_state, [4,5], gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  else:
    temp_state = apply_gate(temp_state, [4,5], gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])))
  k += 1

  if is_pauli == 0:
    temp_state = apply_gate(temp_state, [6,7], gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  else:
    temp_state = apply_gate(temp_state, [6,7], gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])))
  k += 1

  if is_pauli == 0:
    temp_state = apply_gate(temp_state, [5,7], gate = s_gate_double_inverse(theta[len_s * k: len_s * (k+1)]))
  else:
    temp_state = apply_gate(temp_state, [5,7], gate = convert_to_pauli_basis(s_gate_double_inverse(theta[len_s * k: len_s * (k+1)])))
  k += 1

  return temp_state

def ttn_circ(theta, state, is_pauli = 0):
  n_qubits = len(state.shape)
  temp_state = np.copy(state)
  len_s = 4
  k = 2

  if is_pauli == 0:
    temp_state = apply_gate(temp_state, [5,7], gate = s_gate_double(theta[len_s * k: len_s * (k+1)]))
  else:
    temp_state = apply_gate(temp_state, [5,7], gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)])))
  k -= 1

  if is_pauli == 0:
    temp_state = apply_gate(temp_state, [6,7], gate = s_gate_double(theta[len_s * k: len_s * (k+1)]))
  else:
    temp_state = apply_gate(temp_state, [6,7], gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)])))
  k -= 1

  if is_pauli == 0:
    temp_state = apply_gate(temp_state, [4,5], gate = s_gate_double(theta[len_s * k: len_s * (k+1)]))
  else:
    temp_state = apply_gate(temp_state, [4,5], gate = convert_to_pauli_basis(s_gate_double(theta[len_s * k: len_s * (k+1)])))
  k -= 1

  return temp_state



# Inverse Shadow Operator

In this section, we compute the inverse shadow operator.

In [7]:
inv = generate_inv_matrix()

0
10000
20000
30000
40000
50000
60000


# Example

In this ection, we show how to run AISO in different settings. 

## VQSS

### Prepare true state

In [9]:

n_qubits = 8
state = np.reshape(np.array([1] + [0] * ((2 ** n_qubits)-1)), tuple([2] * n_qubits)) # All 0 state
ansatz = 'ttn'
no_of_parameters = 24 # Total number of parameters in the ansatz
#true_theta = np.random.uniform(0, 2 * np.pi, no_of_parameters) # randomly selected true parameters

if ansatz == 'ala':
  true_state = ala_inverse(true_theta, state)
elif ansatz == 'ttn':
  true_state = ttn_inverse(true_theta, state)
elif ansatz == 'mera':
  true_state = mera_inverse(true_theta, state)
elif ansatz == 'hea':
  true_state = hea_inverse(true_theta, state)


### Generate shadows

In [140]:

t1 = time.time()
rho_hat = generate_shallow_shadow(true_state, 3, 10000)
print(time.time() - t1)

Curernt shadow:  999
Curernt shadow:  1999
Curernt shadow:  2999
Curernt shadow:  3999
Curernt shadow:  4999
Curernt shadow:  5999
Curernt shadow:  6999
Curernt shadow:  7999
Curernt shadow:  8999
Curernt shadow:  9999
998.9982144832611


In [11]:
shadow_state = inv * rho_hat

### VQA

In [143]:
number_of_starting_points = 5 # Carry out optimization 'number_of_starting_points' times, each time initialized at a different location.
infid_lists = []
theta_lists = []
for i in range(number_of_starting_points):
    theta = np.random.uniform(0, 2 * np.pi, no_of_parameters)
    c_alpha = 0.4
    a_alpha = 0.4
    no_of_iter = 5000
    ansatz = 'hea'
    infid_list, theta_list = vqa_vqss(theta, true_state, ansatz, c_alpha, a_alpha, no_of_iter, interval = 1, no_of_shots = 0)
    infid_lists.append(infid_list)
    theta_lists.append(theta_lists)

Current iteration: 100
Current Infidelity: 0.9929542807518191
Current iteration: 200
Current Infidelity: 0.9882763346200113
Current iteration: 300
Current Infidelity: 0.9819923499903708
Current iteration: 400
Current Infidelity: 0.9744681173342562
Current iteration: 500
Current Infidelity: 0.9672561441688172
Current iteration: 600
Current Infidelity: 0.9597069518708944
Current iteration: 700
Current Infidelity: 0.9517054873880423
Current iteration: 800
Current Infidelity: 0.9445354917669536
Current iteration: 900
Current Infidelity: 0.9314568946240959
Current iteration: 1000
Current Infidelity: 0.9189845456826288
Current iteration: 1100
Current Infidelity: 0.904170445141591
Current iteration: 1200
Current Infidelity: 0.8843822575110558
Current iteration: 1300
Current Infidelity: 0.8400525989838126
Current iteration: 1400
Current Infidelity: 0.6769100287278706
Current iteration: 1500
Current Infidelity: 0.11362653560271585
Current iteration: 1600
Current Infidelity: 0.010291075003120764

KeyboardInterrupt: 


### AISO

In [13]:
number_of_starting_points = 5 # Carry out optimization 'number_of_starting_points' times, each time initialized at a different location.
infid_lists = []
theta_lists = []
for i in range(number_of_starting_points):
    theta = np.random.uniform(0, 2 * np.pi, len(true_theta))
    c_alpha = 0.4
    a_alpha = 0.4
    no_of_iter = 5000
    ansatz = 'ttn'

    infid_list, theta_list = aiso_vqss(theta, true_state, shadow_state, ansatz, c_alpha, a_alpha, no_of_iter, interval = 100)
    infid_lists.append(infid_list)
    theta_lists.append(theta_list)

Current iteration: 100
Current Infidelity: 0.9997091826438891
Current iteration: 200
Current Infidelity: 0.9999959003752664
Current iteration: 300
Current Infidelity: 0.9998337407243313
Current iteration: 400
Current Infidelity: 0.9996789644859805
Current iteration: 500
Current Infidelity: 0.9995629801498593
Current iteration: 600
Current Infidelity: 0.9994534803635253
Current iteration: 700
Current Infidelity: 0.9993402395927845
Current iteration: 800
Current Infidelity: 0.9989342031668013
Current iteration: 900
Current Infidelity: 0.9984244618807195
Current iteration: 1000
Current Infidelity: 0.9973687862791621
Current iteration: 1100
Current Infidelity: 0.9963644922582628
Current iteration: 1200
Current Infidelity: 0.9946904959283634
Current iteration: 1300
Current Infidelity: 0.9918716096901188
Current iteration: 1400
Current Infidelity: 0.9861640084103266
Current iteration: 1500
Current Infidelity: 0.9672621862079591
Current iteration: 1600
Current Infidelity: 0.8915187560985791
C

KeyboardInterrupt: 

## VQCS

In this section, we mean by circuit state the state that we get when we apply the unknown gate on the second register of a maximally entangled state

### Prepare true circuit state

In [23]:

n_qubits = 8
state = np.reshape(np.array([1] + [0] * ((2 ** n_qubits)-1)), tuple([2] * n_qubits))
state = entangle(state, is_pauli = 0)
ansatz = 'mera'
no_of_parameters = 20
#true_theta = np.random.uniform(0, 2 * np.pi, no_of_parameters)

if ansatz == 'ala':
  true_state = ala_circ_inverse(true_theta, state)
elif ansatz == 'ttn':
  true_state = ttn_circ_inverse(true_theta, state)
elif ansatz == 'mera':
  true_state = mera_circ_inverse(true_theta, state)
elif ansatz == 'hea':
  true_state = hea_circ_inverse(true_theta, state)


### Generate shadows

In [374]:

t1 = time.time()
rho_hat = generate_shallow_shadow(true_state, 3, 10000)
print(time.time() - t1)

Curernt shadow:  999
Curernt shadow:  1999
Curernt shadow:  2999
Curernt shadow:  3999
Curernt shadow:  4999
Curernt shadow:  5999
Curernt shadow:  6999
Curernt shadow:  7999
Curernt shadow:  8999
Curernt shadow:  9999
993.540280342102


In [26]:
shadow_state = inv * rho_hat


### VQA

Whichever ansatz is being used, make sure that it is the same as the ansatz in the code cells below. 

In [27]:
def passable_function(theta):
  temp = mera_circ(theta, true_state)
  temp = disentangle(temp, is_pauli=0)
  ans = compute_expectation_all0_directly(temp, no_of_shots = 10000)
  print('Estimated Infidelity: ', 1-ans)
  return -ans

In [28]:
number_of_starting_points = 5 # Carry out optimization 'number_of_starting_points' times, each time initialized at a different location.
ans_list = []
for i in range(number_of_starting_points):
    ans = minimize(passable_function, 
                          x0 = np.random.uniform(0, 2 * np.pi, len(true_theta)), 
                          method = 'Powell', 
                          options = {'maxfev': 1000})
    
    temp = mera_circ(ans.x, true_state)
    temp = disentangle(temp, is_pauli=0)
    fidelity = compute_expectation_all0_directly(temp, no_of_shots = 0)
    
    print('True Infidelity: ', 1-fidelity)
    ans_list.append(ans)

Estimated Infidelity:  0.9949
Estimated Infidelity:  0.9963
Estimated Infidelity:  0.992
Estimated Infidelity:  0.9929
Estimated Infidelity:  0.9936
Estimated Infidelity:  0.9918
Estimated Infidelity:  0.994
Estimated Infidelity:  0.9899
Estimated Infidelity:  0.9924
Estimated Infidelity:  0.9907
Estimated Infidelity:  0.9924
Estimated Infidelity:  0.9924
Estimated Infidelity:  0.9909
Estimated Infidelity:  0.9943
Estimated Infidelity:  0.9935
Estimated Infidelity:  0.9915
Estimated Infidelity:  0.9913
Estimated Infidelity:  0.9938
Estimated Infidelity:  0.9903
Estimated Infidelity:  0.9929
Estimated Infidelity:  0.9919
Estimated Infidelity:  0.9906
Estimated Infidelity:  0.9912
Estimated Infidelity:  0.994
Estimated Infidelity:  0.9919
Estimated Infidelity:  0.9929
Estimated Infidelity:  0.9938
Estimated Infidelity:  0.9946
Estimated Infidelity:  0.9927
Estimated Infidelity:  0.9912
Estimated Infidelity:  0.9935
Estimated Infidelity:  0.9916
Estimated Infidelity:  0.9925
Estimated Inf

Estimated Infidelity:  0.7472
Estimated Infidelity:  0.7158
Estimated Infidelity:  0.7239
Estimated Infidelity:  0.7045
Estimated Infidelity:  0.7169
Estimated Infidelity:  0.7107
Estimated Infidelity:  0.7133
Estimated Infidelity:  0.7114
Estimated Infidelity:  0.7115
Estimated Infidelity:  0.7114
Estimated Infidelity:  0.7501
Estimated Infidelity:  0.8973
Estimated Infidelity:  0.7138
Estimated Infidelity:  0.7582
Estimated Infidelity:  0.7055
Estimated Infidelity:  0.7178
Estimated Infidelity:  0.7153
Estimated Infidelity:  0.7122999999999999
Estimated Infidelity:  0.7093
Estimated Infidelity:  0.7198
Estimated Infidelity:  0.7066
Estimated Infidelity:  0.7107
Estimated Infidelity:  0.7158
Estimated Infidelity:  0.7208
Estimated Infidelity:  0.7626999999999999
Estimated Infidelity:  0.8842
Estimated Infidelity:  0.7179
Estimated Infidelity:  0.7543
Estimated Infidelity:  0.7165
Estimated Infidelity:  0.7143999999999999
Estimated Infidelity:  0.7158
Estimated Infidelity:  0.7257
Esti

Estimated Infidelity:  0.27349999999999997
Estimated Infidelity:  0.279
Estimated Infidelity:  0.27890000000000004
Estimated Infidelity:  0.2671
Estimated Infidelity:  0.2784
Estimated Infidelity:  0.2694
Estimated Infidelity:  0.26639999999999997
Estimated Infidelity:  0.2701
Estimated Infidelity:  0.27569999999999995
Estimated Infidelity:  0.2711
Estimated Infidelity:  0.27549999999999997
Estimated Infidelity:  0.2713
Estimated Infidelity:  0.278
Estimated Infidelity:  0.2704
Estimated Infidelity:  0.5164
Estimated Infidelity:  0.5508
Estimated Infidelity:  0.2619
Estimated Infidelity:  0.29159999999999997
Estimated Infidelity:  0.3337
Estimated Infidelity:  0.2563
Estimated Infidelity:  0.2603
Estimated Infidelity:  0.2631
Estimated Infidelity:  0.26139999999999997
Estimated Infidelity:  0.2541
Estimated Infidelity:  0.25960000000000005
Estimated Infidelity:  0.2573
Estimated Infidelity:  0.25529999999999997
Estimated Infidelity:  0.2571
Estimated Infidelity:  0.26959999999999995
Es

Estimated Infidelity:  0.14990000000000003
Estimated Infidelity:  0.1532
Estimated Infidelity:  0.14839999999999998
Estimated Infidelity:  0.3043
Estimated Infidelity:  0.6331
Estimated Infidelity:  0.15280000000000005
Estimated Infidelity:  0.25139999999999996
Estimated Infidelity:  0.1664
Estimated Infidelity:  0.14629999999999999
Estimated Infidelity:  0.1442
Estimated Infidelity:  0.15069999999999995
Estimated Infidelity:  0.14370000000000005
Estimated Infidelity:  0.14629999999999999
Estimated Infidelity:  0.14890000000000003
Estimated Infidelity:  0.1471
Estimated Infidelity:  0.14049999999999996
Estimated Infidelity:  0.14790000000000003
Estimated Infidelity:  0.1441
Estimated Infidelity:  0.30179999999999996
Estimated Infidelity:  0.6478999999999999
Estimated Infidelity:  0.1542
Estimated Infidelity:  0.2582
Estimated Infidelity:  0.1582
Estimated Infidelity:  0.14939999999999998
Estimated Infidelity:  0.14339999999999997
Estimated Infidelity:  0.15259999999999996
Estimated Inf


### AISO

Whichever ansatz is being used, make sure that it is the same as the ansatz in the code cells below. 

In [385]:
obs = maximally_entangled_pauli()
def passable_function(theta):
  temp = mera_circ_inverse(theta, obs, is_pauli = 1)
  ans = val = np.dot(temp.flatten(), shadow_state)
  
  print('Estimated Infidelity: ', 1-ans)
  return -ans

In [29]:
number_of_starting_points = 5 # Carry out optimization 'number_of_starting_points' times, each time initialized at a different location.
ans_list = []
for i in range(number_of_starting_points):
    ans = minimize(passable_function, 
                          x0 = np.random.uniform(0, 2 * np.pi, len(true_theta)), 
                          method = 'Powell', 
                          options = {'maxfev': 1000})
    
    temp = mera_circ(ans.x, true_state)
    temp = disentangle(temp, is_pauli=0)
    fidelity = compute_expectation_all0_directly(temp, no_of_shots = 0)
    
    print('True Infidelity: ', 1-fidelity)
    
    ans_list.append(ans)

Estimated Infidelity:  0.9875
Estimated Infidelity:  0.9878
Estimated Infidelity:  0.9906
Estimated Infidelity:  0.9953
Estimated Infidelity:  0.9879
Estimated Infidelity:  0.9907
Estimated Infidelity:  0.9908
Estimated Infidelity:  0.9911
Estimated Infidelity:  0.988
Estimated Infidelity:  0.9894
Estimated Infidelity:  0.989
Estimated Infidelity:  0.989
Estimated Infidelity:  0.9901
Estimated Infidelity:  0.9904
Estimated Infidelity:  0.9872
Estimated Infidelity:  0.9898
Estimated Infidelity:  0.99
Estimated Infidelity:  0.9902
Estimated Infidelity:  0.9898
Estimated Infidelity:  0.9907
Estimated Infidelity:  0.9882
Estimated Infidelity:  0.9907
Estimated Infidelity:  0.9898
Estimated Infidelity:  0.9897
Estimated Infidelity:  0.9905
Estimated Infidelity:  0.9896
Estimated Infidelity:  0.992
Estimated Infidelity:  0.9842
Estimated Infidelity:  0.9929
Estimated Infidelity:  0.9868
Estimated Infidelity:  0.9879
Estimated Infidelity:  0.9873
Estimated Infidelity:  0.987
Estimated Infidel

Estimated Infidelity:  0.6662
Estimated Infidelity:  0.6565
Estimated Infidelity:  0.6976
Estimated Infidelity:  0.6504
Estimated Infidelity:  0.6561
Estimated Infidelity:  0.6514
Estimated Infidelity:  0.6548
Estimated Infidelity:  0.6461
Estimated Infidelity:  0.6483
Estimated Infidelity:  0.6487
Estimated Infidelity:  0.6549
Estimated Infidelity:  0.9786
Estimated Infidelity:  0.6468
Estimated Infidelity:  0.8945
Estimated Infidelity:  0.5217
Estimated Infidelity:  0.49660000000000004
Estimated Infidelity:  0.5025
Estimated Infidelity:  0.504
Estimated Infidelity:  0.5005999999999999
Estimated Infidelity:  0.5069
Estimated Infidelity:  0.4982
Estimated Infidelity:  0.49570000000000003
Estimated Infidelity:  0.4958
Estimated Infidelity:  0.5009
Estimated Infidelity:  0.49250000000000005
Estimated Infidelity:  0.5800000000000001
Estimated Infidelity:  0.7855
Estimated Infidelity:  0.49839999999999995
Estimated Infidelity:  0.5628
Estimated Infidelity:  0.5054000000000001
Estimated Inf

Estimated Infidelity:  0.09450000000000003
Estimated Infidelity:  0.058699999999999974
Estimated Infidelity:  0.07269999999999999
Estimated Infidelity:  0.0615
Estimated Infidelity:  0.061000000000000054
Estimated Infidelity:  0.05469999999999997
Estimated Infidelity:  0.05989999999999995
Estimated Infidelity:  0.06220000000000003
Estimated Infidelity:  0.059699999999999975
Estimated Infidelity:  0.26670000000000005
Estimated Infidelity:  0.5637
Estimated Infidelity:  0.05669999999999997
Estimated Infidelity:  0.15039999999999998
Estimated Infidelity:  0.09189999999999998
Estimated Infidelity:  0.06059999999999999
Estimated Infidelity:  0.0776
Estimated Infidelity:  0.06440000000000001
Estimated Infidelity:  0.06010000000000004
Estimated Infidelity:  0.06389999999999996
Estimated Infidelity:  0.060699999999999976
Estimated Infidelity:  0.05789999999999995
Estimated Infidelity:  0.056499999999999995
Estimated Infidelity:  0.05900000000000005
Estimated Infidelity:  0.05559999999999998
Es

Estimated Infidelity:  0.030399999999999983
Estimated Infidelity:  0.02959999999999996
Estimated Infidelity:  0.02849999999999997
Estimated Infidelity:  0.029000000000000026
Estimated Infidelity:  0.030399999999999983
Estimated Infidelity:  0.02859999999999996
Estimated Infidelity:  0.029299999999999993
Estimated Infidelity:  0.028800000000000048
Estimated Infidelity:  0.025800000000000045
Estimated Infidelity:  0.02839999999999998
Estimated Infidelity:  0.025699999999999945
Estimated Infidelity:  0.028299999999999992
Estimated Infidelity:  0.029000000000000026
Estimated Infidelity:  0.02749999999999997
Estimated Infidelity:  0.02959999999999996
Estimated Infidelity:  0.031299999999999994
Estimated Infidelity:  0.030100000000000016
Estimated Infidelity:  0.029900000000000038
Estimated Infidelity:  0.027000000000000024
Estimated Infidelity:  0.031000000000000028
Estimated Infidelity:  0.030000000000000027
Estimated Infidelity:  0.027800000000000047
Estimated Infidelity:  0.0281000000000

Estimated Infidelity:  0.22760000000000002
Estimated Infidelity:  0.5371
Estimated Infidelity:  0.011800000000000033
Estimated Infidelity:  0.11170000000000002
Estimated Infidelity:  0.042200000000000015
Estimated Infidelity:  0.011700000000000044
Estimated Infidelity:  0.012700000000000045
Estimated Infidelity:  0.016700000000000048
Estimated Infidelity:  0.0121
Estimated Infidelity:  0.010399999999999965
Estimated Infidelity:  0.013599999999999945
Estimated Infidelity:  0.012700000000000045
Estimated Infidelity:  0.012599999999999945
Estimated Infidelity:  0.012299999999999978
Estimated Infidelity:  0.22809999999999997
Estimated Infidelity:  0.5274
Estimated Infidelity:  0.011499999999999955
Estimated Infidelity:  0.1068
Estimated Infidelity:  0.04349999999999998
Estimated Infidelity:  0.013299999999999979
Estimated Infidelity:  0.027900000000000036
Estimated Infidelity:  0.01319999999999999
Estimated Infidelity:  0.014399999999999968
Estimated Infidelity:  0.013900000000000023
Estim